# Visão Geral do Notebook: Extração e Classificação de Marcadores Imunohistoquímicos de Mama com LLM

Este notebook implementa um pipeline completo para extração automatizada de informações de laudos médicos de câncer de mama, utilizando **Large Language Models (LLMs)** integrados ao ambiente Databricks. O objetivo principal é identificar e classificar marcadores imunohistoquímicos essenciais para a definição de subtipos moleculares do câncer de mama, facilitando análises clínicas e epidemiológicas.

---

## Principais Bibliotecas e Frameworks Utilizados

- **PySpark**: Para processamento distribuído de grandes volumes de dados, manipulação de DataFrames e execução de queries SQL.
- **Databricks Foundation Models**: Uso do modelo LLM `databricks-llama-4-maverick` via função `ai_query` para extração de informações textuais.
- **Pandas e NumPy**: Manipulação e análise de dados tabulares em memória local.
- **OpenAI**: Integração com APIs de modelos de linguagem (quando necessário).
- **Regex (re)**: Expressões regulares para extração heurística de marcadores (validação).
- **openpyxl**: Exportação e formatação de relatórios em Excel.
- **octoops**: Framework para monitoramento e alertas (integração com Sentinel).
- **MLflow**: Rastreamento de experimentos e modelos.
- **Jinja2**: Geração dinâmica de templates de prompt para o LLM.

---

## Fluxo de Trabalho/Etapas Principais

1. **Configuração do Ambiente**: Instalação de dependências e inicialização do Spark.
2. **Extração de Dados**: Query SQL incremental sobre a tabela `refined.saude_preventiva.fleury_laudos`, filtrando apenas laudos de mama, pacientes do sexo feminino e exames relevantes (`IH-NEO`, `IHMAMA`).
3. **Processamento via LLM**: Geração de prompts dinâmicos e extração dos marcadores imunohistoquímicos usando o LLM da Databricks.
4. **Classificação Molecular**: Aplicação de regras clínicas para classificar os casos em subtipos como Luminal A, Luminal B, HER-2 Superexpresso, Triplo Negativo, etc.
5. **Validação Heurística**: Implementação de funções regex para extração heurística dos mesmos marcadores, permitindo comparação e cálculo de métricas de acurácia.
6. **Exportação e Auditoria**: Geração de relatórios Excel com formatação condicional e persistência dos resultados em tabela Delta Lake.

---

## Dados Envolvidos

- **Tabela de Origem**: `refined.saude_preventiva.fleury_laudos`
- **Tabela de Destino**: `refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico`
- **Colunas Importantes**:
  - `laudo_tratado`: Texto do laudo médico (input para o LLM)
  - `sigla_exame`, `linha_cuidado`, `sexo_cliente`: Filtros de seleção
  - `id_ficha`, `id_item`, `id_subitem`: Chaves para merge/upsert
  - `_datestamp`: Controle de incrementalidade

---

## Funções e Componentes de Destaque

- **prompt_laudo_template()**: Gera o template de prompt para o LLM, detalhando os critérios de extração dos marcadores.
- **Funções de Extração Heurística**: `extrai_receptor_estrogeno`, `extrai_receptor_progesterona`, `extrai_status_her2`, `extrai_ki67_percentual`, `extrai_status_ck5_6`.
- **avalia_extracao_sem_ground_truth_imuno**: Compara a extração do LLM com a heurística, campo a campo.
- **agrega_resultados_dinamico**: Consolida métricas de acurácia por campo.
- **configura_excel**: Gera planilha Excel com formatação condicional para auditoria dos resultados.

---

## Resultados/Saídas Esperadas

- **DataFrame Estruturado**: Com marcadores extraídos e classificação molecular.
- **Planilha Excel**: Relatório de validação com destaques visuais para erros.
- **Tabela Delta Atualizada**: Persistência dos resultados processados para uso futuro e integração com outros sistemas.

---

## Pré-requisitos

- Ambiente Databricks com acesso ao Spark e Foundation Models.
- Permissões de leitura/escrita nas tabelas de origem e destino.
- Instalação dos pacotes Python necessários (`octoops`, `openpyxl`, `jinja2`, etc.).

---

## Considerações Importantes

- **Incrementalidade**: O pipeline processa apenas registros novos, evitando retrabalho.
- **Validação Dupla**: Combinação de LLM e heurística aumenta a confiabilidade dos resultados.
- **Custo Computacional**: O uso de LLMs pode ser custoso; recomenda-se limitar o volume de dados em testes.
- **Flexibilidade**: O pipeline pode ser adaptado para outros tipos de laudos ou marcadores, bastando ajustar o template de prompt e as funções heurísticas.

---

# Extração de dado - Imunohistoquimico
**Extrair os seguintes labels estruturados:**
- Receptor de Estrógeno (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE ESTRÓGENO - POSITIVO " 
  - "RECEPTOR DE ESTRÓGENO – NEGATIVO" 

- Receptor de Progesterona (categórica): POSITIVO ou NEGATIVO 
  - "RECEPTOR DE PROGESTERONA - POSITIVO" 
  - "RECEPTOR DE PROGESTERONA - NEGATIVO" 

- Status do HER-2 (categórica): Negativo, inconclusivo ou positivo 
  - "HER-2 - ESCORE 0" = Negativo 
  - "HER-2 - ESCORE 1+" = Negativo 
  - "HER-2  -  ESCORE  2+" = Inconclusivo 
  - "HER-2  -  ESCORE  3+" = Positivo 

- Porcentagem do Ki-67 (numérica):  "KI-67 - POSITIVO EM 20% DAS CÉLULAS NEOPLÁSICAS"
  - Deve ser extraído esse número da porcentagem nessa frase 

- Status CK5/6 (categórica): POSITIVO ou NEGATIVO 
  - "CK5/6 - POSITIVO "ou "CK5/6 - NEGATIVO" 
  
+ Para laudos que não possuem carcinoma, ou seja, casos negativos para câncer não devem se processados no LLM.

## Instalação de Dependências

Esta célula instala o pacote `octoops`, que é utilizado para monitoramento e alertas no ambiente Databricks. As outras dependências necessárias (como openai, tqdm, pandas e databricks-feature-store) estão comentadas pois provavelmente já estão instaladas no ambiente ou serão instaladas por outro processo.

O pacote Octoops permite configurar alertas e notificações em caso de falhas no pipeline, sendo uma peça importante na infraestrutura de monitoramento do processo de extração de dados.

In [0]:
# %pip install openai -q
# %pip install tqdm -q
# %pip install pandarallel -q
# %pip install databricks-feature-store -q
# %pip install ace_tools -q
%pip install octoops openpyxl jinja2

## Reinicialização do Python

Após a instalação de novas dependências, é necessário reiniciar o interpretador Python para garantir que as bibliotecas recém-instaladas estejam disponíveis no ambiente de execução. O comando `dbutils.library.restartPython()` realiza essa reinicialização, garantindo que todas as dependências estejam carregadas corretamente antes de prosseguir com a execução do notebook.

In [0]:
dbutils.library.restartPython()

## Importação de Bibliotecas e Configuração Inicial

Esta célula realiza a importação de todas as bibliotecas necessárias para o processamento dos laudos e inicializa o ambiente Spark.

**Objetivos principais:**
1. Importar bibliotecas para manipulação de dados (PySpark, pandas, numpy)
2. Importar bibliotecas para processamento de texto e regex
3. Importar ferramentas para logging e monitoramento (mlflow)
4. Importar bibliotecas para integração com APIs externas (openai)
5. Inicializar a sessão Spark
6. Obter o token de autenticação Databricks

**Bibliotecas principais:**
- **PySpark**: Framework para processamento distribuído de dados
- **pandas/numpy**: Ferramentas para manipulação e análise de dados
- **openai**: Cliente para comunicação com APIs de modelos de linguagem
- **mlflow**: Plataforma para gerenciamento do ciclo de vida de modelos de ML
- **tqdm**: Biblioteca para barras de progresso
- **re**: Biblioteca para processamento de expressões regulares

A sessão Spark é configurada com o nome "LLM_Extractor", e o token Databricks é obtido do contexto do notebook para autenticação com serviços externos.

In [0]:
from pyspark.sql import SparkSession
import ast
import json
import re
import os
import sys
import mlflow
import time
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Any
import openai
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from databricks.feature_store import FeatureStoreClient
from pyspark.sql import Row



spark = SparkSession.builder.appName("LLM_Extractor").getOrCreate()

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get() if dbutils.notebook.entry_point.getDbutils().notebook().getContext() is not None else None

## Desativação de Exibição MLflow no Notebook

Esta célula desativa a exibição automática de informações do MLflow no notebook, configurando o comportamento do MLflow para evitar que ele adicione visualizações e saídas extras durante o rastreamento de experimentos. Isto é particularmente útil em notebooks de produção onde queremos controlar precisamente o que é exibido, sem poluição visual extra criada pelo rastreamento automático do MLflow.

O comando `mlflow.tracing.disable_notebook_display()` evita que artefatos, métricas e parâmetros sejam automaticamente exibidos no notebook quando são registrados, mantendo a saída limpa e focada apenas no que está sendo explicitamente exibido pelo código.

In [0]:
# mlflow.tracing.disable_notebook_display()

## Configuração dos Parâmetros de Consulta

Esta célula define os parâmetros de consulta para extrair laudos de imunohistoquímica específicos para câncer de mama. São configurados:

1. **Tabela de Destino** (`table_imuno`): Define o nome da tabela Delta onde os resultados serão armazenados.

2. **Cláusula WHERE para Incremento** (`where_clause`): Implementa uma estratégia de carga incremental, buscando apenas registros com timestamp mais recente que o último carregamento na tabela de destino.

3. **Filtros de Extração** (`filtro_extracao`): Define critérios específicos para selecionar apenas laudos relevantes:
   - Linha de cuidado específica para mama
   - Pacientes do sexo feminino
   - Siglas de exame específicas de imunohistoquímica (IH-NEO, IHMAMA)
   - Presença dos termos "mama" e "carcinoma" nos laudos

Esses filtros garantem que apenas os laudos de imunohistoquímica relevantes para câncer de mama feminino sejam processados, otimizando o uso de recursos computacionais e do modelo de linguagem.

In [0]:
table_imuno = "refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico" 

where_clause = f"""
WHERE
    flr.`_datestamp` >= (
        SELECT MAX(imuno._datestamp)
        FROM {table_imuno} imuno
    )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(sexo_cliente) = 'F'
        AND sigla_exame IN ("IH-NEO", "IHMAMA")
        AND laudo_tratado RLIKE '(?i)mama' AND laudo_tratado RLIKE '(?i)carcinoma'
"""

## Execução da Consulta SQL e Carregamento dos Dados

Esta célula constrói e executa uma consulta SQL para extrair laudos de exames imunohistoquímicos relevantes para análise. 

**Processo detalhado:**

1. **Construção da Consulta**: A consulta SQL é construída usando um Common Table Expression (CTE) chamado `base` que:
   - Seleciona colunas relevantes da tabela principal de laudos (`refined.saude_preventiva.fleury_laudos`)
   - Aplica o filtro incremental definido em `where_clause` para obter apenas registros novos
   - Usa a cláusula `filtro_extracao` para filtrar apenas laudos de mama com carcinoma

2. **Seleção de Campos**: A consulta extrai informações essenciais como:
   - Identificadores (id_marca, id_unidade, id_cliente, id_ficha, etc.)
   - Datas (dth_pedido, dth_resultado)
   - Tipo de exame (sigla_exame)
   - Conteúdo do laudo (laudo_tratado)
   - Informações da linha de cuidado e sexo do cliente

3. **Execução da Consulta**: A consulta é executada através do Spark SQL, criando um DataFrame `df_spk`

4. **Visualização**: O DataFrame resultante é exibido na interface para verificação inicial dos dados carregados

Esta etapa é fundamental para preparar o conjunto de dados que será processado pelo modelo de linguagem nas etapas subsequentes.

In [0]:
query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`
    FROM refined.saude_preventiva.fleury_laudos flr    
    {where_clause}
)
SELECT * FROM base
{filtro_extracao}
"""
df_spk = spark.sql(query)
display(df_spk)

## Verificação de Sexo e Tipos de Exame

Esta célula executa uma verificação básica dos dados carregados, exibindo os valores distintos para duas colunas críticas: `sexo_cliente` e `sigla_exame`. 

O objetivo desta verificação é:
1. Confirmar que apenas pacientes do sexo feminino (F) foram incluídos no dataset, conforme especificado no filtro
2. Verificar quais siglas de exames de imunohistoquímica estão presentes nos dados carregados
3. Validar que os filtros da consulta SQL foram aplicados corretamente

Esta etapa de validação rápida ajuda a garantir a qualidade dos dados antes de prosseguir com análises mais detalhadas e o processamento pelo modelo de linguagem.

In [0]:
display(df_spk.select("sexo_cliente", "sigla_exame").distinct())

## Contagem de Registros

Esta célula simples realiza a contagem do número total de registros no DataFrame `df_spk` carregado pela consulta SQL. O comentário "# 3635" indica que, em uma execução anterior, foram encontrados 3.635 registros.

Conhecer o volume de dados é importante para:
1. Estimar o tempo total de processamento pelo modelo de linguagem
2. Verificar se o volume está de acordo com o esperado
3. Avaliar a necessidade de processamento em lotes ou amostragem para testes

Esta contagem serve como referência para monitoramento do pipeline em execuções futuras.

In [0]:
df_spk.count()

In [0]:
def prompt_laudo_template() -> str:
    # REMOVER O 'f' ANTES DAS ASPAS TRIPLAS
    # ESCAPAR O '%' LITERAL COM '%%'
    prompt = """A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

    Laudo clínico:
    \"\"\"{laudo_texto}\"\"\"

    ### Critérios de extração:

    - **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"

    - **Ki-67 (%%)**: retorne o valor numérico da porcentagem de positividade do KI-67 entre aspas, como uma **string**, caso seja um intervalo de valores retorne o valor máximo.

    - **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

    ### Saída esperada (dicionário Python válido):
    ```python
    {
    "receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
    "ki67_percentual": float |  0,
    "status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
    }
    """ 
    return prompt.strip()
    

from pyspark.sql.types import StructType, StructField, StringType # Importações necessárias

# --- Schema para o JSON de saída do LLM (usado por from_json) ---
# Este schema define a estrutura esperada do JSON retornado pelo LLM
llm_output_schema = StructType([
StructField("receptor_estrogeno", StringType(), True),
StructField("receptor_progesterona", StringType(), True),
StructField("status_her2", StringType(), True),
StructField("ki67_percentual", StringType(), True), # Manter como StringType para lidar com "NÃO INFORMADO" ou intervalos
StructField("status_ck5_6", StringType(), True),
])

## Funções de Extração Heurística (Pseudo-Gold)

Esta célula define um conjunto de funções para extração heurística de informações dos laudos médicos, utilizando expressões regulares. Estas extrações servem como um "pseudo-gold standard" para avaliar o desempenho do LLM.

### Funções de Extração Individual

1. **`extrai_receptor_estrogeno`**: Identifica o status do receptor de estrogênio
   - Reconhece padrões como "receptor de estrogênio: positivo"
   - Identifica abreviações como "ER+" e "ER-"

2. **`extrai_receptor_progesterona`**: Identifica o status do receptor de progesterona
   - Reconhece padrões como "receptor de progesterona: positivo" 
   - Identifica abreviações como "PR+" e "PR-"

3. **`extrai_status_her2`**: Determina o status do HER-2 baseado em escores
   - Processa padrões como "HER-2 ESCORE X+"
   - Mapeia escores 0/1+ como "NEGATIVO", 2+ como "INCONCLUSIVO", 3+ como "POSITIVO"

4. **`extrai_ki67_percentual`**: Extrai o valor percentual do Ki-67
   - Processa padrões como "Ki-67: 20%"
   - Caso o resultado seja uma faixa de valores, extrai o valor máximo da faixa

5. **`extrai_status_ck5_6`**: Identifica o status do CK5/6
   - Reconhece padrões como "CK5/6 - POSITIVO"
   - Identifica abreviações como "CK5/6+" e "CK5/6-"

### Função de Avaliação

**`avalia_extracao_sem_ground_truth_imuno`**: Função chave que compara extrações do LLM com extrações heurísticas
   - Gera extrações heurísticas para todos os campos
   - Compara campo a campo os resultados do modelo vs. heurísticas
   - Produz métricas de avaliação por campo (acertou/não acertou)

Este sistema de avaliação permite medir a qualidade das extrações do LLM sem necessidade de anotação manual de dados, sendo uma ferramenta crucial para validação contínua do pipeline.

In [0]:
# =================================================================
# Funções de extração heurística (pseudo-gold) para o novo prompt
# =================================================================

def extrai_receptor_estrogeno(txt: str) -> str:
    """
    Extrai o status de Receptor de Estrogênio: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    """

    # Padrões comuns: "receptor de estrogênio: positivo" ou "er+: positivo" etc.
    # Adicionado flags=re.IGNORECASE para ignorar maiúsculas/minúsculas em toda a string
    # Ajustado [eê]genio para [eê]g[eê]nio|ogeno para cobrir todas as variações de "estrogênio" e "estrogeno"
    # estr[oó]g[eê]\w+
    if re.search(r"receptor\s+de\s+estr[oó]g(?:[eê]nio|eno)\s*[:\-]?\s*positivo", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"receptor\s+de\s+estr[oó]g(?:[eê]nio|eno)\s*[:\-]?\s*negativo", txt, flags=re.IGNORECASE):
        return "NEGATIVO"

    # Abreviações: "ER+" / "ER-" (estes já usavam re.IGNORECASE, então não precisam de alteração)
    if re.search(r"\ber\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\ber\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"

    return "NÃO INFORMADO"

def extrai_receptor_progesterona(txt: str) -> str:
    """
    Extrai o status de Receptor de Progesterona: valores possíveis
    "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    Ajustado para capturar padrões como "RECEPTOR DE PROGESTERONA - ESCORE 1+ (NEGATIVO)".
    """
    txt_lower = txt.lower()

    # Define um limite de caracteres para a busca do status após "receptor de progesterona".
    # Este valor deve ser ajustado com base na variabilidade dos seus relatórios.
    # Deve ser suficiente para cobrir um resultado direto (ex: "RP - ESCORE 1+ (NEGATIVO)")
    # e pequeno o suficiente para não pular para seções de referência distantes.
    max_chars_between_rp_and_status = 45 # Um valor como 50-100 caracteres é um bom ponto de partida.

    # Padrão para "receptor de progesterona ... positivo"
    # O `.{0,X}?` permite texto intermediário de forma não-gananciosa.
    if re.search(
        r"receptor\s+de\s+progesterona[*]?(.{0," + str(max_chars_between_rp_and_status) + r"}?)positivo\b",
        txt_lower
    ):
        return "POSITIVO"

    # Padrão para "receptor de progesterona ... negativo"
    # O `.{0,X}?` permite texto intermediário de forma não-gananciosa.
    if re.search(
        r"receptor\s+de\s+progesterona[*]?(.{0," + str(max_chars_between_rp_and_status) + r"}?)negativo\b",
        txt_lower
    ):
        return "NEGATIVO"

    # Abreviações: "PR+" / "PR-"
    # Estes são mais específicos e não precisam da limitação de distância tão complexa.
    # Mantemos a busca no 'txt' original com IGNORECASE para estas abreviações.
    if re.search(r"\bpr\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bpr\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"

    return "NÃO INFORMADO"
    

    # Abreviações: "PR+" / "PR-"
    # Estes são mais específicos e não precisam da limitação de distância tão complexa.
    # Mantemos a busca no 'txt' original com IGNORECASE para estas abreviações.
    if re.search(r"\bpr\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bpr\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"

    return "NÃO INFORMADO"

def extrai_status_her2(txt: str) -> str:
    """
    Extrai o Status do HER-2 baseado em termos de score:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"
    - Novos formatos: "HER-2 - POSITIVO", "HER-2 - NEGATIVO"
    """
    txt_lower = txt.lower()

    # Padrão para "HER-2 - POSITIVO"
    if re.search(r"her[-\s]?2\s*[:\-]?\s*positivo\b", txt_lower, flags=re.IGNORECASE):
        return "POSITIVO"

    # Padrão para "HER-2 - NEGATIVO"
    if re.search(r"her[-\s]?2\s*[:\-]?\s*negativo\b", txt_lower, flags=re.IGNORECASE):
        return "NEGATIVO"

    # Primeiro, busca padrão "her-2 ... escore X+"
    m = re.search(r"her[-\s]?2.*?s?core\s*[:\-]?\s*([0-3]\+?)", txt_lower, flags=re.IGNORECASE)
    if m:
        score = m.group(1)
        if score.startswith("0") or score == "1+":
            return "NEGATIVO"
        if score == "2+":
            return "INCONCLUSIVO"
        if score == "3+":
            return "POSITIVO"

    # Outra forma: "her2 3+" isolado
    m2 = re.search(r"\bher[-]?2\s*[:\-]?\s*([0-3]\+)\b", txt_lower, flags=re.IGNORECASE)
    if m2:
        score2 = m2.group(1)
        if score2 == "1+":
            return "NEGATIVO"
        if score2 == "2+":
            return "INCONCLUSIVO"
        if score2 == "3+":
            return "POSITIVO"

    return "NÃO INFORMADO"

def extrai_ki67_percentual(txt: str) -> float:
    """
    Extrai o valor de Ki-67 em porcentagem.
    Exemplo no texto: "Ki-67: 20%", "Ki 67 15 %", etc.
    Se não encontrar, retorna 0.0.
    Ajustado para evitar a captura de percentuais de outras seções (e.g., valores de referência).
    """
    txt_lower = txt.lower()

    # Define um limite de caracteres para a busca do percentual após "KI67".
    # Este valor deve ser ajustado com base na variabilidade dos seus relatórios.
    # Deve ser suficiente para cobrir um resultado direto (ex: "KI67 - POSITIVO EM 10% DAS CÉLULAS")
    # e pequeno o suficiente para não pular para seções de referência distantes.
    # Um valor como 50-100 caracteres deve ser razoável para a maioria dos casos.
    max_chars_between_ki67_and_percent = 100 # Ajuste conforme a necessidade real dos seus dados

    # O padrão busca "ki67", então *qualquer coisa* (não-ganancioso)
    # até o limite de 'max_chars_between_ki67_and_percent', e então o percentual.
    m = re.search(
        r"ki[-\s]?67.{0," + str(max_chars_between_ki67_and_percent) + r"}?[:\-]?\s*(\d{1,3}(?:[.,]\d+)?)\s*%",
        txt_lower,
        flags=re.IGNORECASE
    )
    if m:
        try:
            # Substitui vírgula por ponto para garantir a conversão correta para float
            percent_str = m.group(1).replace(',', '.')
            return float(percent_str)
        except ValueError:
            # Em caso de erro na conversão (improvável com o regex atual, mas boa prática)
            return 0.0
    return 0.0

def extrai_status_ck5_6(txt: str) -> str:
    """
    Extrai o status do CK5/6: "POSITIVO", "NEGATIVO" ou retorna "NÃO INFORMADO".
    Esta versão mantém a verificação de distância e resolve o problema de ambiguidade.
    """
    txt_lower = txt.lower()

    # 1. Verificar abreviações (CK5/6+ ou CK5/6-) - Estes são os mais específicos e diretos.
    # Mantemos a busca no 'txt' original com IGNORECASE para estas abreviações.
    if re.search(r"\bck5\s*\/\s*6\s*[\+]\b", txt, flags=re.IGNORECASE):
        return "POSITIVO"
    if re.search(r"\bck5\s*\/\s*6\s*[\-]\b", txt, flags=re.IGNORECASE):
        return "NEGATIVO"

    # 2. Procurar por "CK5/6" e o status mais próximo a ele.
    # max_chars_after_ck5_6 é a sua verificação de distância!
    # Ela define o quão longe o regex pode "olhar" após encontrar "CK5/6".
    max_chars_after_ck5_6 = 70 # Ajuste este valor conforme a variabilidade dos seus relatórios

    # O padrão busca "ck5/6", então *qualquer coisa* (não-ganancioso)
    # até o limite de 'max_chars_after_ck5_6', e então o *primeiro* "negativo" ou "positivo".
    match = re.search(
        r"ck5\s*\/\s*6.{0," + str(max_chars_after_ck5_6) + r"}?(\bnegativo\b|\bpositivo\b)",
        txt_lower
    )

    if match:
        status_found = match.group(1) # Captura o que foi matched no primeiro grupo (negativo ou positivo)
        if status_found == "positivo":
            return "POSITIVO"
        elif status_found == "negativo":
            return "NEGATIVO"

    # Se nenhuma das condições acima foi atendida
    return "NÃO INFORMADO"

# ==========================================================================
# Função de avaliação sem ground truth completo (novo conjunto de campos)
# ==========================================================================

def avalia_extracao_sem_ground_truth_imuno(laudo_texto: str, json_modelo: dict):
    """
    Gera pseudo-gold (json_heu) para os campos do novo prompt
    e compara com json_modelo (saída da IA).
    Retorna:
      - json_heu: dicionário com valores heurísticos
      - comparacoes: dicionário que, para cada campo, traz:
          * valor_heu
          * valor_mod
          * acertou (boolean)
    """
    # 1. Gera pseudo-gold (json_heu)
    rei = extrai_receptor_estrogeno(laudo_texto)
    rpr = extrai_receptor_progesterona(laudo_texto)
    her = extrai_status_her2(laudo_texto)
    ki6 = extrai_ki67_percentual(laudo_texto)
    ck6 = extrai_status_ck5_6(laudo_texto)

    json_heu = {
        "receptor_estrogeno": rei,
        "receptor_progesterona": rpr,
        "status_her2": her,
        "ki67_percentual": ki6,
        "status_ck5_6": ck6
    }

    # 2. Prepara json_modelo: se não for dict, converte para dict vazio
    if not isinstance(json_modelo, dict):
        json_modelo = {}

    # 3. Comparações campo a campo
    comparacoes = {}

    # Campos categóricos (strings)
    for campo in ["receptor_estrogeno", "receptor_progesterona", "status_her2", "status_ck5_6"]:
        val_heu = json_heu[campo]
        val_mod = json_modelo.get(campo, "NÃO INFORMADO")
        comparacoes[campo] = {
            "valor_heu": val_heu,
            "valor_mod": val_mod,
            "acertou": (val_heu == val_mod)
        }

    # Campo Ki-67 (%)
    val_heu_ki = json_heu["ki67_percentual"]
    try:
        val_mod_ki = float(json_modelo.get("ki67_percentual", 0))
    except (ValueError, TypeError):
        val_mod_ki = 0.0
    comparacoes["ki67_percentual"] = {
        "valor_heu": val_heu_ki,
        "valor_mod": val_mod_ki,
        "acertou": (val_heu_ki == val_mod_ki)
    }

    return json_heu, comparacoes

## Função de Agregação de Resultados

Esta célula define uma função especializada para agregar resultados de avaliação de múltiplos laudos, calculando métricas de desempenho para cada campo extraído.

**Objetivo da Função:**
`agrega_resultados_dinamico` processa uma lista de comparações (gerada pela função `avalia_extracao_sem_ground_truth_imuno`) e calcula estatísticas agregadas, incluindo:
- Número total de acertos por campo
- Total de laudos analisados
- Taxa de acerto (acertos/total) para cada campo

**Detalhamento da Implementação:**
1. Utiliza `Counter` para contabilizar os acertos por campo de forma eficiente
2. Itera por todas as comparações e todos os campos dentro de cada comparação
3. Verifica o atributo "acertou" de cada campo para contabilizar os acertos
4. Gera um dicionário estruturado com as estatísticas de cada campo
5. Garante que todos os campos estão representados no resultado, mesmo os que não tiveram acertos

**Características Importantes:**
- É uma função genérica que não assume um conjunto fixo de campos, podendo processar qualquer estrutura de comparações
- Trata adequadamente o caso de divisão por zero quando não há laudos
- Mantém consistência na estrutura de saída para facilitar análises posteriores

Esta função é crucial para avaliar quantitativamente o desempenho do modelo de extração em todo o conjunto de dados.

In [0]:
from collections import Counter

def agrega_resultados_dinamico(lista_comparacoes):
    """
    Agraga resultados de uma lista de dicionários de comparações, retornando para cada campo:
      - acertos: número de vezes que 'acertou' == True
      - total: número total de laudos (len(lista_comparacoes))
      - taxa_acerto: acertos / total (ou 0.0 se total == 0)
    
    Suporta qualquer conjunto de chaves em cada dict, desde que cada valor seja outro dict contendo a chave 'acertou'.
    """
    total_laudos = len(lista_comparacoes)
    acertos_por_campo = Counter()

    # Para cada comparação, percorremos todas as chaves e contamos os acertos
    for comp in lista_comparacoes:
        for campo, info in comp.items():
            # Supondo que info seja um dict com a chave "acertou"
            if info.get("acertou", False):
                acertos_por_campo[campo] += 1

    # Monta o dicionário de saída
    resultado = {}
    for campo, acertos in acertos_por_campo.items():
        resultado[campo] = {
            "acertos": acertos,
            "total": total_laudos,
            "taxa_acerto": (acertos / total_laudos) if total_laudos > 0 else 0.0
        }

    # É possível que exista algum campo em alguma comparação que nunca teve 'acertou' == True.
    # Se quisermos incluir também esses campos (com acertos = 0), podemos varrer as chaves da primeira entrada:
    if lista_comparacoes:
        primeira = lista_comparacoes[0]
        for campo in primeira.keys():
            if campo not in resultado:
                resultado[campo] = {
                    "acertos": 0,
                    "total": total_laudos,
                    "taxa_acerto": 0.0
                }

    return resultado

## Verificação de Contagem de Registros

Esta célula executa uma verificação rápida de quantos registros estão disponíveis para processamento no DataFrame `df_spk`. 

Esta contagem é importante para:
1. Verificar se temos dados para processar
2. Dimensionar o tempo de execução esperado
3. Avaliar se o processo de filtragem nos retornou um conjunto de dados compatível com o esperado

O valor retornado será o número total de laudos de imunohistoquímica de mama com menção a carcinoma que estão disponíveis para processamento pelo modelo de linguagem.

In [0]:
df_spk.count()

## Extração de Marcadores com LLM e Classificação Molecular

Esta célula implementa o core do pipeline de processamento dos laudos, envolvendo a extração de informações estruturadas dos laudos médicos utilizando um modelo de linguagem e a subsequente classificação molecular dos casos de câncer. Este é o ponto central do notebook, onde o processamento efetivo dos dados acontece.

**Objetivo:**
1. Configurar o cliente LLM para comunicação com o endpoint Databricks
2. Processar os laudos médicos para extração de biomarcadores 
3. Classificar os tumores em subtipos moleculares com base nos biomarcadores extraídos

**Fluxo de Processamento:**
1. **Inicialização do Cliente LLM**: Configura o cliente OpenAI para se comunicar com o endpoint Databricks
2. **Definição de Persona**: Define o papel do modelo como "médico oncologista especialista"
3. **Amostragem de Dados**: Limita a análise a 15 registros para processamento local
4. **Processamento em Lote**: Envia os laudos para o modelo em lotes
5. **Pós-processamento**: Converte as respostas do modelo em formato estruturado
6. **Classificação Molecular**: Aplica regras específicas para determinar o subtipo molecular:
   - **Luminal A**: ER+ ou PR+, HER2-, Ki-67 < 14%
   - **Luminal B**: ER+ ou PR+, HER2-, Ki-67 ≥ 14%
   - **Luminal com HER2 Positivo**: ER+ ou PR+, HER2+
   - **HER-2 Superexpresso**: ER-, PR-, HER2+
   - **Triplo Negativo**: ER-, PR-, HER2-

A célula contém também código comentado que representa versões anteriores ou alternativas do pipeline, incluindo opções para persistência dos dados e registro de métricas, que podem ser descomentadas conforme necessário.

# Extração via LLM, Parsing e Classificação Molecular

Esta célula implementa o núcleo do pipeline de extração e classificação dos marcadores imunohistoquímicos dos laudos médicos utilizando um modelo de linguagem (LLM) distribuído no Databricks.

**Principais etapas e lógica:**

- **Importação de funções e tipos Spark:** São importados tipos de dados (`StructType`, `StringType`, `DoubleType`, etc.) e funções para manipulação de DataFrames (`from_json`, `col`), além do pandas para eventuais operações locais.
- **Verificação de existência de dados:** O bloco só é executado se o DataFrame `df_spk` possuir registros.
- **Preparação do prompt:** O template do prompt é ajustado para uso com a função SQL `FORMAT_STRING`, permitindo injetar o texto do laudo em cada chamada ao LLM.
- **Geração do prompt final:** Uma nova coluna é criada com o prompt específico para cada laudo, garantindo que o modelo receba o contexto correto.
- **Chamada distribuída ao LLM:** Utiliza a função `ai_query` para enviar os prompts ao modelo `databricks-llama-4-maverick`, recebendo as respostas em formato JSON.
- **Limpeza da resposta do LLM:** Remove marcadores de bloco de código (```python, ```) e espaços extras para garantir que o JSON seja parseável.
- **Parsing das respostas:** Converte a resposta limpa do LLM em colunas estruturadas, extraindo os campos de interesse (`receptor_estrogeno`, `receptor_progesterona`, `status_her2`, `ki67_percentual`, `status_ck5_6`).
- **Depuração:** Exibe as respostas parseadas para inspeção.
- **Conversão do Ki-67 para float:** Cria uma coluna numérica para facilitar a classificação molecular.
- **Classificação molecular:** Aplica regras clínicas para categorizar cada caso em subtipos como Luminal A, Luminal B, Luminal com HER2 Positivo, HER-2 Superexpresso, Triplo Negativo ou Indefinido, com base nos marcadores extraídos.

**Impacto:** Ao final, o DataFrame resultante contém todos os marcadores extraídos e a classificação molecular de cada laudo, pronto para validação, exportação ou persistência.

**Exemplo de regra de classificação:**
```python
F.when(
    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
    (F.col("status_her2") == "NEGATIVO") &
    (F.col("ki67_percentual_float") < 14),
    "Luminal A"
)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, IntegerType
from pyspark.sql.functions import from_json, col
from pyspark.sql.functions import col # F já está importado como F.col
import pandas as pd

# --- Bloco principal de execução (dentro do `if df_spk.count() > 0:`) ---

if df_spk.count() > 0:
    # Nome do modelo Foundation Model no Databricks
    llm_model_name = "databricks-llama-4-maverick" # Ou o modelo que você estiver usando

    # Preparamos o template do prompt para ser usado com a função SQL FORMAT_STRING.
    # O placeholder para o laudo será '%s'.
    # Precisamos escapar as aspas internas do template para que ele seja uma string SQL válida.
    # O `json.dumps` faz isso para nós.
    # A string resultante será algo como: "A seguir está um laudo... \"\"\"%s\"\"\" ..."
    prompt_template_for_sql = json.dumps(prompt_laudo_template().replace('"""{laudo_texto}"""', '"""%s"""'))

    # --- ADIÇÃO PARA DEPURAR: Verifique o prompt final enviado ao LLM ---
    # Crie uma coluna temporária para o prompt final para inspecionar
    # Usamos F.expr com FORMAT_STRING para injetar o laudo_tratado
    df_with_prompts = df_spk.withColumn(
        "final_prompt_for_llm",
        F.expr(f"FORMAT_STRING({prompt_template_for_sql}, laudo_tratado)")
    )
    # print("--- df_with_prompts (mostrando o prompt final enviado ao LLM) ---")
    # df_with_prompts.select("laudo_tratado", "final_prompt_for_llm").display()
    # --- FIM DA ADIÇÃO ---

    # Usa ai_query para invocar o LLM de forma distribuída
    df_with_llm_raw_responses = df_with_prompts.withColumn(
        "llm_raw_response",
        F.expr(f"""
            ai_query(
                '{llm_model_name}',
                TO_JSON(MAP(
                    'prompt', final_prompt_for_llm,
                    'temperature', 0.0,
                    'max_tokens', 4000,
                    'top_p', 0.75,
                    'frequency_penalty', 0.0,
                    'presence_penalty', 0.0
                )),
                'JSON'
            )
        """)
    )

    # --- ADIÇÃO PARA LIMPAR A SAÍDA BRUTA DO LLM ANTES DO from_json ---
    # Remove os marcadores de bloco de código "```python" e "```"
    df_cleaned_llm_responses = df_with_llm_raw_responses.withColumn(
        "llm_cleaned_response",
        F.regexp_replace(F.col("llm_raw_response"), "```python|```", "") # Remove ambos os marcadores
    )
    # Remove espaços em branco extras no início e fim
    df_cleaned_llm_responses = df_cleaned_llm_responses.withColumn(
        "llm_cleaned_response",
        F.trim(F.col("llm_cleaned_response"))
    )

    # print("--- df_cleaned_llm_responses (mostrando a saída do LLM após limpeza) ---")
    # df_cleaned_llm_responses.select("laudo_tratado", "llm_raw_response", "llm_cleaned_response").display()
    # --- FIM DA ADIÇÃO ---

    # Converte as strings JSON das respostas do LLM em colunas estruturadas
    # AGORA USANDO A COLUNA LIMPA: "llm_cleaned_response"
    df_llm_parsed = df_cleaned_llm_responses.withColumn(
        "llm_parsed_output", F.from_json(F.col("llm_cleaned_response"), llm_output_schema)
    ).select(
        "*", # Mantém todas as colunas originais
        F.col("llm_parsed_output.receptor_estrogeno").alias("receptor_estrogeno"),
        F.col("llm_parsed_output.receptor_progesterona").alias("receptor_progesterona"),
        F.col("llm_parsed_output.status_her2").alias("status_her2"),
        F.col("llm_parsed_output.ki67_percentual").alias("ki67_percentual"),
        F.col("llm_parsed_output.status_ck5_6").alias("status_ck5_6")
    ).drop("llm_raw_response", "llm_parsed_output", "llm_cleaned_response") # Remove as colunas intermediárias




    # --- ADIÇÃO PARA DEPURAR: Verifique a saída bruta do LLM ---
    # print("--- df_with_llm_raw_responses (mostrando a saída bruta do LLM) ---")
    # df_with_llm_raw_responses.select("laudo_tratado", "llm_raw_response").display()
    # --- FIM DA ADIÇÃO ---


    # --- ADIÇÃO PARA DEPURAR: Verifique a saída parseada do LLM ---
    print("--- df_llm_parsed (mostrando a saída parseada do LLM) ---")
    df_llm_parsed.select("laudo_tratado", "receptor_estrogeno", "receptor_progesterona", "status_her2", "ki67_percentual", "status_ck5_6").display()
    # --- FIM DA ADIÇÃO ---








    # Garante que ki67_percentual seja float para a classificação
    df_llm_parsed = df_llm_parsed.withColumn(
        "ki67_percentual_float",
        F.when(
            F.col("ki67_percentual").cast(DoubleType()).isNotNull(),
            F.col("ki67_percentual").cast(DoubleType())
        ).otherwise(0.0) # Se não for um número válido, assume 0.0
    )

    # print("DF com respostas LLM e parsing")
    # display(df_llm_parsed)

    # --- Sua lógica de classificação final (mantida) ---
    df_final_classif = df_llm_parsed.withColumn(
            "categoria_final",
            F.when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual_float") < 14)
                ),
                "Luminal A"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "NEGATIVO") &
                    (F.col("ki67_percentual_float") >= 14)
                ),
                "Luminal B"
            ).when(
                (
                    ((F.col("receptor_estrogeno") == "POSITIVO") | (F.col("receptor_progesterona") == "POSITIVO")) &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "Luminal com HER2 Positivo"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "POSITIVO")
                ),
                "HER-2 Superexpresso"
            ).when(
                (
                    (F.col("receptor_estrogeno") == "NEGATIVO") &
                    (F.col("receptor_progesterona") == "NEGATIVO") &
                    (F.col("status_her2") == "NEGATIVO")
                ),
                "Triplo Negativo"
            ).otherwise("Indefinido")
        )

    # print("df_final_classif")
    # display(df_final_classif)

## Processamento de Métricas de Avaliação (Código de Referência)

Esta célula contém código de referência para processar métricas de avaliação do desempenho do modelo de extração. Embora não seja executado diretamente (pois depende de variáveis não definidas no fluxo principal), este código serve como template para avaliação de qualidade.

O código demonstra como:
1. Criar um DataFrame de métricas com laudos e resultados
2. Converter estruturas de dados em formatos apropriados
3. Executar a função de avaliação para cada laudo
4. Transformar os resultados para facilitar análise

Este tipo de avaliação é útil para monitorar a qualidade das extrações e identificar áreas para melhoria do prompt ou do processamento.

In [0]:
# --- Bloco de avaliação de métricas (após a geração de df_llm_parsed ou df_final_classif) ---

# Para realizar a avaliação, você precisará coletar uma amostra dos dados para o driver.
# Isso é aceitável para avaliação, mas não para o pipeline de inferência principal.
# Ajuste o `limit(N)` conforme a quantidade de dados que você deseja avaliar.
df_sample_for_eval = df_llm_parsed.toPandas() # Coleta uma amostra para avaliação local

lista_laudos_eval = df_sample_for_eval["laudo_tratado"].tolist()

# Prepara a saída do LLM para a função de avaliação
# O `json_model_output` deve ser um dicionário Python para cada linha
lista_json_modelo_eval = []
for index, row in df_sample_for_eval.iterrows():
    lista_json_modelo_eval.append({
        "receptor_estrogeno": row["receptor_estrogeno"],
        "receptor_progesterona": row["receptor_progesterona"],
        "status_her2": row["status_her2"],
        "ki67_percentual": row["ki67_percentual"], # Use a string original do LLM
        "status_ck5_6": row["status_ck5_6"]
    })

lista_pseudo_gold_eval = []
lista_comparacoes_eval = []

# As funções avalia_extracao_sem_ground_truth_imuno e agrega_resultados_dinamico
# devem estar definidas em células anteriores.
for laudo_txt, json_mod in zip(lista_laudos_eval, lista_json_modelo_eval):
    json_heu, comp = avalia_extracao_sem_ground_truth_imuno(laudo_txt, json_mod)
    lista_pseudo_gold_eval.append(json_heu)
    lista_comparacoes_eval.append(comp)

# Agrega os resultados das comparações
json_metricas = agrega_resultados_dinamico(lista_comparacoes_eval)

print("Métricas de Avaliação:")
print(json.dumps(json_metricas, indent=2))

# Opcional: Para visualizar os resultados detalhados da avaliação
# df_metrics_eval = pd.DataFrame()
# df_metrics_eval["laudos"] = lista_laudos_eval
# df_metrics_eval["json_modelo"] = lista_json_modelo_eval
# df_metrics_eval["json_heuristico"] = lista_pseudo_gold_eval
# df_metrics_eval["comparacoes"] = lista_comparacoes_eval
# display(df_metrics_eval)



## Visualização das Métricas de Avaliação

Esta célula exibe o DataFrame `df_metrics`, que contém os resultados detalhados da avaliação das extrações do modelo em comparação com as extrações heurísticas (pseudo-gold).

O DataFrame exibe:
1. Os laudos médicos originais
2. Para cada campo extraído (receptores de estrogênio, progesterona, status HER2, etc.):
   - O valor extraído pelo método heurístico
   - O valor extraído pelo modelo de linguagem
   - Um indicador booleano de acerto (True/False)

Esta visualização é fundamental para análise detalhada do desempenho do modelo campo a campo e laudo a laudo, permitindo identificar padrões de erros específicos e oportunidades de melhoria.

In [0]:
# Opcional: Para visualizar os resultados detalhados da avaliação
df_metrics_eval = pd.DataFrame()
df_metrics_eval["laudos"] = lista_laudos_eval
df_metrics_eval["json_modelo"] = lista_json_modelo_eval
df_metrics_eval["json_heuristico"] = lista_pseudo_gold_eval
df_metrics_eval["comparacoes"] = lista_comparacoes_eval
display(df_metrics_eval)


## Contagem de Registros Classificados

Esta célula realiza uma contagem simples do número de registros no DataFrame `df_final_classif`, que contém os laudos processados com suas respectivas classificações moleculares.

Esta contagem serve para:
1. Verificar quantos laudos foram processados com sucesso
2. Confirmar que o processamento ocorreu como esperado
3. Fornecer informação importante para monitoramento do pipeline

O número retornado representa a quantidade total de laudos de imunohistoquímica de mama que foram classificados em subtipos moleculares pelo pipeline.

In [0]:
df_final_classif.count()

## Persistência dos Dados Processados em Delta Lake

Esta célula final implementa a persistência dos dados processados em uma tabela Delta Lake, para uso posterior em análises e modelos preditivos. Utiliza o padrão de operação "merge" (upsert) para garantir idempotência e evitar duplicações.

**Componentes principais:**
1. **Configurações de Monitoramento**: Definição do webhook para alertas via Sentinel
2. **Inicialização da Sessão Spark**: Configuração da sessão para operações com Delta
3. **Definição do Caminho de Saída**: Especificação da tabela de destino
4. **Função `insert_data`**: Encapsula a lógica de inserção/atualização:
   - Verifica se a tabela Delta já existe
   - Cria a tabela se necessário
   - Executa uma operação de merge quando a tabela já existe
   
5. **Tratamento de Erros**: Estrutura try/except com:
   - Verificação da existência de dados para inserção
   - Chamada à função de inserção quando há dados
   - Envio de notificação via Sentinel quando não há dados para processamento
   - Captura e relato de exceções

Esta persistência é essencial para que os dados estruturados extraídos dos laudos e as classificações moleculares possam ser utilizados por sistemas downstream, como dashboards de BI, modelos preditivos ou ferramentas de apoio à decisão clínica.

In [0]:
# import traceback
# from octoops import Sentinel
# from delta.tables import DeltaTable

# WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

# # spark = SparkSession.builder.appName("DfPandasparaSpark").getOrCreate() # REMOVIDO: SparkSession já é inicializada no Databricks

# # OUTPUT_DATA_PATH = dbutils.widgets.get("OUTPUT_DATA_PATH") # Mantido se for usado via widget
# OUTPUT_DATA_PATH = "refined.saude_preventiva.fleury_laudos_mama_imunohistoquimico"

# # função para salvar dados na tabela
# # Renomeado o parâmetro para maior clareza
# def insert_data(df_to_save, output_data_path):

#     # Cria a tabela Delta se não existir
#     if not DeltaTable.isDeltaTable(spark, output_data_path):
#         df_to_save.write.format("delta").saveAsTable(output_data_path)
#     else:
#         # Carrega a tabela Delta existente
#         delta_table = DeltaTable.forPath(spark, output_data_path)

#         # Faz o merge (upsert)
#         (delta_table.alias("target")
#         .merge(
#             df_to_save.alias("source"), # Usando df_to_save
#             "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
#         )
#         .whenMatchedUpdateAll() #atualiza todos os campos se o ID já existir
#         .whenNotMatchedInsertAll() #insere se o ID não existir
#         .execute())

# # salvar dados na tabela
# try:
#     # Calcula a contagem uma única vez para otimização
#     num_records_to_save = df_final_classif.count()

#     if num_records_to_save > 0:
#         # Inserir tabela catalog
#         insert_data(df_final_classif, OUTPUT_DATA_PATH)
#         print(f'Total de registros salvos na tabela: {num_records_to_save}')
#     else:
#         # A mensagem de erro é mais específica para o caso de não haver laudos
#         error_message = "Fleury Imunuhistoquimico - Não há laudos para extração ou processamento."
#         sentinela_ds_ai_business = Sentinel(
#             project_name='Monitor_Linhas_Cuidado_Mama',
#             env_type=WEBHOOK_DS_AI_BUSINESS_STG,
#             task_title='Fleury Mama Imunuhistoquimico'
#         )
#         sentinela_ds_ai_business.alerta_sentinela(
#             categoria='Alerta',
#             mensagem=error_message,
#             job_id_descritivo='4_fleury_mama_Imunuhistoquimico'
#         )
# except Exception as e:
#     traceback.print_exc()
#     raise e


In [0]:
# --- Funções auxiliares para exportação de Excel (manter estas funções em uma célula) ---
def salvar_excel(df, nome_arquivo):
    """
    Salva um DataFrame em um arquivo Excel.

    Args:
        df (pandas.DataFrame): DataFrame a ser salvo.
        nome_arquivo (str): Nome do arquivo Excel.
    """

    # After installing, save the DataFrame to Excel again
    df.to_excel(nome_arquivo, index=False)

def conta_marcadores(txt: str, marcador: str) -> int:
    """
    Conta quantas vezes uma marcador aparece no texto, ignorando maiúsculas/minúsculas.
    """
    marcadores = {
        'estrogenio': r"\b(receptor\s+de\s+estr[oó]g(?:[eê]nio|eno))\b",
        'progesterona': r"\b(receptor\s+de\s+progesterona)\b",
        'her2': r"\bher2\b",
        'ck5_6': r"\bck5\s*\/\s*6\b",
        'ki67': r"\bki[-\s]?67\b"
    }
    pattern = marcadores.get(marcador)
    if not pattern:
        raise ValueError(f"marcador '{marcador}' não reconhecida. Use uma das seguintes: {list(marcadores.keys())}")

    matches = re.findall(pattern, txt, flags=re.IGNORECASE)
    return len(matches)

def pega_texto_marcador(txt: str, sigla: str) -> str: # Alterado para retornar str, não int
    """
    Pega o texto que vem depois do marcador, limitado em 100 caracteres.
    """
    siglas = {
        'estrogenio': r"\b(receptor\s+de\s+estr[oó]g(?:[eê]nio|eno))\b",
        'progesterona': r"\b(receptor\s+de\s+progesterona)\b",
        'her2': r"\bher2\b",
        'ck5_6': r"\bck5\s*\/\s*6\b",
        'ki67': r"\bki[-\s]?67\b"
    }
    pattern = siglas.get(sigla)
    if not pattern:
        raise ValueError(f"sigla '{sigla}' não reconhecida. Use uma das seguintes: {list(siglas.keys())}")

    match = re.search(pattern, txt, flags=re.IGNORECASE)
    if match:
        # Pega o texto após o marcador, limitado a 100 caracteres
        start = match.end()
        end = start + 100
        return txt[start:end].strip()
    return ""


def configura_excel(df, nome_arquivo):
    """
    Configura o DataFrame para salvar em Excel com as seguintes regras:
    - Nas colunas booleanas (receptor_estrogeno.acertou, receptor_progesterona.acertou, status_her2.acertou, status_ck5_6.acertou, ki67_percentual.acertou), colorir de vermelho se False.
    - Na coluna (erro), colorir de vermelho se True.
    - Salvar em uma aba chamada 'Resultados' e outra aba chamada 'Resumo' com a contagem de erros e o percentual de erros por coluna em relação ao total de linhas, inserir uma linha com o total de erros e do percentual.
    - Salvar o arquivo com o nome '{nome_arquivo}'
    """

    import pandas as pd

    # Função para aplicar a formatação condicional
    def colorir_vermelho(val):
        color = 'red' if val == False else ''
        return f'background-color: {color}'
    
    # Função para aplicar a formatação condicional
    def colorir_erro(val):
        color = 'red' if val == True else ''
        return f'background-color: {color}'
    
    # def colorir_verde(val):
    #     color = 'green' if val == True else ''
    #     return f'background-color: {color}'

    # Seleciona as colunas booleanas para aplicar a formatação
    colunas_booleanas = [
        'receptor_estrogeno.acertou',
        'receptor_progesterona.acertou',
        'status_her2.acertou',
        'status_ck5_6.acertou',
        'ki67_percentual.acertou',
    ]

    # Cria um objeto Styler
    styler = df.style

    # Aplica a formatação condicional nas colunas booleanas
    for coluna in colunas_booleanas:
        if coluna in df.columns:
            styler = styler.map(colorir_vermelho, subset=[coluna])
    
    # Aplica a formatação condicional na coluna 'erro'
    if 'erro' in df.columns:
        styler = styler.map(colorir_erro, subset=['erro'])

    # Cria o resumo de erros por coluna
    resumo = {coluna: df[coluna].value_counts().get(False, 0) for coluna in colunas_booleanas}
    resumo_df = pd.DataFrame(list(resumo.items()), columns=['Coluna', 'Contagem de Erros'])
    resumo_df['Percentual de Erros (%)'] = (resumo_df['Contagem de Erros'] / len(df)) * 100
    resumo_df.loc['Total'] = resumo_df.sum(numeric_only=True)

    # Salva em um arquivo Excel com duas abas
    with pd.ExcelWriter(f'{nome_arquivo}', engine='openpyxl') as writer:
        styler.to_excel(writer, sheet_name='Resultados', index=False)
        resumo_df.to_excel(writer, sheet_name='Resumo', index=False)

    print(f"Arquivo Excel salvo como {nome_arquivo} com as abas 'Resultados' e 'Resumo'.")
    

In [0]:
# --- Bloco para gerar a planilha de comparação (em uma nova célula) ---

# df_sample_for_eval e lista_comparacoes_eval devem ter sido gerados na célula de avaliação anterior.
# Se a célula de avaliação não for executada, este bloco falhará.

if 'df_sample_for_eval' in locals() and not df_sample_for_eval.empty:
    # Cria uma cópia do DataFrame de amostra para não modificar o original
    df_validacao = df_sample_for_eval[['laudo_tratado']].copy()

    # Normaliza a lista de comparações para adicionar as colunas de acerto
    # lista_comparacoes_eval deve vir da célula de avaliação
    if 'lista_comparacoes_eval' in locals() and lista_comparacoes_eval:
        resultados_expandidos = pd.json_normalize(lista_comparacoes_eval)
        
        # Adiciona um prefixo para evitar conflito de nomes de colunas
        # resultados_expandidos = resultados_expandidos.add_prefix('comparacao_')
        df_validacao = pd.concat(
            [df_validacao.reset_index(drop=True), resultados_expandidos.reset_index(drop=True)],
            axis=1
        )

        # Contagem de quantas vezes cada sigla aparece no laudo
        siglas = ['estrogenio', 'progesterona', 'her2', 'ck5_6', 'ki67']
        for sigla in siglas:
            df_validacao['ctg_' + sigla] = df_validacao['laudo_tratado'].apply(lambda x: conta_marcadores(x, sigla))
            # Adiciona o texto do marcador para análise
            df_validacao['txt_' + sigla] = df_validacao['laudo_tratado'].apply(lambda x: pega_texto_marcador(x, sigla))

        # Insere coluna de erro genérico
        df_validacao['erro'] = df_validacao.apply(lambda x:
                                        (x.get('ki67_percentual.acertou', False) == False) or
                                        (x.get('receptor_estrogeno.acertou', False) == False) or
                                        (x.get('receptor_progesterona.acertou', False) == False) or
                                        (x.get('status_her2.acertou', False) == False) or
                                        (x.get('status_ck5_6.acertou', False) == False), axis=1)
        
        nome_arquivo_excel = "df_validacao_laura.xlsx"

        # Aplica formatação condicional e configura as cores das céluas
        configura_excel(df_validacao, nome_arquivo_excel)
                
        display(df_validacao.head(3))
        print(df_validacao.columns)

    else:
        print("Lista de comparações vazia ou não encontrada. Não foi possível gerar a planilha de comparação.")
else:
    print("DataFrame de amostra para avaliação (df_sample_for_eval) vazio ou não encontrado. Não foi possível gerar a planilha de comparação.")

In [0]:

df_final_classif.describe()